In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
train = train.rename(columns={'3_labels':'label','comment':'text'})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
test = test.rename(columns={'3_labels':'label','comment':'text'})
test.fillna(0, inplace=True)
print(len(test))



1650


In [3]:
train_change_feautre = train.iloc[:, 4:]
test_change_feautre = test.iloc[:, 4:]

train_code_change = train_change_feautre.values
test_code_change = test_change_feautre.values

In [4]:
# !pip install setfit
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [5]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [6]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['Corrective','Adaptive','Perfective'], sample_size=8)

In [7]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [8]:
len(train)

143

In [9]:
train['label'].value_counts()

label
Corrective    48
Perfective    48
Adaptive      47
Name: count, dtype: int64

In [10]:
test['label'].value_counts()

label
Corrective    555
Perfective    552
Adaptive      543
Name: count, dtype: int64

In [11]:
len(train_dataset)

167

In [12]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "precision": precision_score,"recall": recall_score, "f1":f1_score}

In [13]:
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [14]:
train

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
884,6a85160cfa4f99167f29a1323d0175703fb57816,camel,Fixed test--git-svn-id: https://svn.apache.org...,Corrective,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1644,17fdbe6c71b97305d5c1d8dd78a6c7b249c9e481,tomakehurst$wiremock,More flexibility with request body matching\n,Perfective,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
445,2f3d64b70fbd92d4f3fe132bf0dd46a822ee14d6,drools,JBRULES-976: fixing problems with collections-...,Perfective,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
211,2eeb4ebd8c3af6f5838efe510da4c92ceb0ceb35,elasticsearch,improve memcached test--,Perfective,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
402,ec559db68e1ad306b1ba97283fbda1074fa50eb0,hadoop,HDFS-1480. All replicas of a block can end up ...,Corrective,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,0b3db7cbf5eada6176af266cc6bebfcd1332f115,restlet-framework-java,- Additional WADL refactorings.--,Perfective,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1411,f2757568dac2a6473f054101f80ac2b8f291ce4e,Vala,glib-2.0: add GLib.HashTable.foreach\n\nFixes ...,Adaptive,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1759,d64e6060d9ddcdd681fa0e739e16549f344dbc11,iee$iee,some refactoring: move all translator function...,Perfective,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
1021,1f9fb52c7c71ad74b48797f1655ca5b7636e77c3,Valadoc,driver/0.18.x: Add support for vala-0.17.1+\n,Adaptive,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [16]:
from commitfit import CommitFitModel
model = CommitFitModel.from_pretrained(model_id)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [17]:
from commitfit import CommitFitTrainer

trainer = CommitFitTrainer(
    model=model,
    train_dataset=train_dataset,
    train_code_change = train_change_feautre,
    test_code_change = test_change_feautre,
    eval_dataset=test_dataset,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)

In [18]:
%%time
trainer.train()
fewshot_metrics = trainer.evaluate()
fewshot_metrics

Generating Training Pairs: 100%|██████████| 20/20 [00:00<00:00, 74.50it/s] 
***** Running training *****
  Num examples = 6680
  Num epochs = 1
  Total optimization steps = 418
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [01:09<00:00, 69.51s/it]
***** Running evaluation *****


CPU times: user 1min 41s, sys: 27.1 s, total: 2min 8s
Wall time: 1min 11s


{'accuracy': 0.7387878787878788,
 'precision': 0.7397088263293768,
 'recall': 0.7387878787878788,
 'f1': 0.7389233148480663}